In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cvolmft4
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cvolmft4
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10742 sha256=655723c0d2f4841dd76e45e772f96358a5fda83aba7888a669bf1eb2bb64257b
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ap1mvmj/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmp7p1xdfkc".


In [ ]:
git clone https://github.com/nothings/stb.git \
cp stb/stb_image.h /usr/local/include/ \
cp stb/stb_image_write.h /usr/local/include/

fatal: destination path 'stb' already exists and is not an empty directory.


In [ ]:
##The dataset is loaded to your GDrive so need to be mounted
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/Parallel Computing Labs/Lab 4')

Mounted at /content/drive


### **Requirement**

A) A cuda program is required to carry out a 3D convolution over RGB images and save the output ones, the program is given a path to a folder containing the input images and that of an output folder that should contain the outputs, respectively as command line arguments.

1.   kernel1: basic implementation (no tiling)
2.   kernel2: tiling where each block matches the input tile size.
3.   kernel3: tiling where each block matches the output tile size.

Notes:
*   Add necessary paddings so that the output image size is the same as that of the input one.

*   The kernel should be able to handle a batch of images at a time, the batch size is passed as the 3rd argument.
*   The mask is given in a .txt file, whose path is passed as the 4th argument. The first line contains its dimension n (one number only as it's a square mask) then the consecutive n lines contain the mask rows, each row in a separate line. Repeat the mask 3 times for the 3 channels of the image.

  Ex: ./a.out input_folder_path output_folder_path 4 mask.txt

B) Implement the same program in python, using the built-in convolution functions in Pytorch.

C) Profile each program carefully and do sufficient experiments to compare between them and collect insightful results. Organise your results in a tabular form and prepare a comprehensive report explaining all of your findings. Also mention the impact of declaring the mask as constant in terms of execution time and elaborate on this in your report.

In [ ]:
# Mean filter 5x5
%%writefile "mask.txt"
5
0.25  0.25  0.25  0.25  0.25
0.25  0.25  0.25  0.25  0.25
0.25  0.25  0.25  0.25  0.25
0.25  0.25  0.25  0.25  0.25
0.25  0.25  0.25  0.25  0.25

Overwriting mask.txt


In [ ]:
# Mean filter 3x3
%%writefile "mask.txt"
3
0.1111111111111111 0.1111111111111111 0.1111111111111111
0.1111111111111111 0.1111111111111111 0.1111111111111111
0.1111111111111111 0.1111111111111111 0.1111111111111111

Overwriting mask.txt


In [ ]:
# Mean filter 3x3
%%writefile "mask.txt"
3
1/9 1/9 1/9
1/9 1/9 1/9
1/9 1/9 1/9

Overwriting mask.txt


In [ ]:
# edge detection
%%writefile "mask.txt"
3
-1 0 1
-2 0 2
-1 0 1

Overwriting mask.txt


In [ ]:
# Laplacian Operator
%%writefile "mask.txt"
3
0 1 0
1 -4 1
0 1 0

Overwriting mask.txt


In [ ]:
# Edge detection
%%writefile "mask.txt"
3
0 -1 0
-1 5 -1
0 -1 0

Overwriting mask.txt


# Kernel 1 - Basic (No tilling)

In [ ]:
%%writefile "K1_1_17_2_14.cu"
// nvcc -w K1_1_17_2_14.cu -o kernel1.out
// ./kernel1.out Input Output 4 mask.txt

#include <algorithm>
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <stdlib.h>
#include <string.h>
#include <float.h>
#include "dirent.h"

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

#define MAX_MASK_WIDTH 25
__constant__ float c_mask[MAX_MASK_WIDTH];

// Kernel function for 3D convolution with tiling
__global__ void convolution3D(const uint8_t *input, int width, int height, int depth, float *output, float * mask, int maskWidth)
{
    int Col = blockIdx.x * blockDim.x + threadIdx.x;
    int Row = blockIdx.y * blockDim.y + threadIdx.y;
    int batch_index = blockIdx.z;
    int image_index_in_batch = batch_index * width * height * depth;

    if (Col < width && Row < height)
    {
        float pixelValue = 0.0f;
        for(int Channel = 0; Channel < depth; ++Channel)
        {
            for (int j = 0; j < maskWidth; ++j)
            {
                for (int k = 0; k < maskWidth; ++k)
                {
                    int currRow = Row - maskWidth / 2 + j;
                    int currCol = Col - maskWidth / 2 + k;
                    if (currRow > -1 && currRow < height && currCol > -1 && currCol < width)
                    {
                        int maskIndex = j * maskWidth + k;
                        int inputIndex = (currRow * width + currCol) * depth + Channel + image_index_in_batch;
                        pixelValue += static_cast<float>(input[inputIndex]) * c_mask[maskIndex];
                    }
                }
            }
          }
          output[(Row * width + Col) +  batch_index * width * height] = pixelValue;
    }
}



void process_batch(const char *input_folder, const char *output_folder, float *mask, int maskWidth, int batch_size);
void launch_convolution_kernel(std::vector<uint8_t*> images, int width, int height, int depth, float *output_imgs, float *mask, int maskWidth, int batch_size, int batch_index, int current_batch_size);
void load_images(std::vector<uint8_t*>& images, const char *input_folder, int batch_size, int &width, int &height, int &depth) ;
float *createMask(float** maskVal, int maskWidth);
void normalizeFloatImage(float *img, uint8_t *output_img, int width, int height);

int main(int argc, char **argv)
{
    if (argc < 5)
    {
        std::cerr << "Usage: " << argv[0] << " <input_folder> <output_folder> <batch_size> <mask_file>" << std::endl;
        return 1;
    }


    // Load input images from folder
    const char* inputFolder = argv[1];
    const char* outputFolder = argv[2];
    int batch_size = std::atoi(argv[3]);
    const char* maskfilePath = argv[4];
    std::ifstream inFile(maskfilePath); // Open the mask file
    if (!inFile) {
        std::cerr << "Unable to open file mask.txt" << std::endl;
        return 1;
    }

    int maskWidth;
    inFile >> maskWidth; // Read mask width from the first line

    float** maskVal = new float*[maskWidth]; // Allocate memory for rows
    for (int i = 0; i < maskWidth; ++i) {
        maskVal[i] = new float[maskWidth]; // Allocate memory for columns
    }


    // Read mask values from the file
    for (int i = 0; i < maskWidth; ++i) {
        for (int j = 0; j < maskWidth; ++j) {
            std::string maskValue;
            if (!(inFile >> maskValue)) {
                std::cerr << "Error reading mask value" << std::endl;
                return 1;
            }

            // Parse the fraction if it contains '/'
            size_t pos = maskValue.find('/');
            if (pos != std::string::npos) {
                int numerator = std::atoi(maskValue.substr(0, pos).c_str());
                int denominator = std::atoi(maskValue.substr(pos + 1).c_str());
                if (denominator == 0) {
                    std::cerr << "Invalid denominator in mask value" << std::endl;
                    return 1;
                }
                maskVal[i][j] = static_cast<double>(numerator) / denominator;
            } else {
                // If there's no '/', parse the value directly
                maskVal[i][j] = std::atof(maskValue.c_str());
            }
        }
    }
    inFile.close();

    // Create mask using the provided function
    float *mask = createMask(maskVal, maskWidth);

    process_batch(inputFolder, outputFolder, mask, maskWidth, batch_size);

    delete[] mask;
    return 0;
}

void process_batch(const char *input_folder, const char *output_folder, float *mask, int maskWidth, int batch_size)
{
    // Load images from input folder
    int width, height, depth;
    std::vector<uint8_t*> images;
    std::vector<uint8_t*> imagesOutput;
    load_images(images, input_folder, batch_size, width, height, depth);

    int outputImageSize = width * height;
    int number_of_images = images.size();
    int num_iterations = (number_of_images + batch_size - 1) / batch_size;
    int remaining_images = number_of_images;
    uint8_t* output_img_uint8;
    float *output_imgs = (float *)malloc(batch_size * outputImageSize * sizeof(float));

    for (int batch_index = 0; batch_index < num_iterations; ++batch_index)
    {
        // Start convolution batch processing
        int current_batch_size = min(batch_size, remaining_images);
        printf("Started processing %d images with dimensions : (%d, %d, %d) \n", current_batch_size, width, height, depth);
        launch_convolution_kernel(images, width, height, depth, output_imgs, mask, maskWidth, batch_size, batch_index, current_batch_size);
        remaining_images -= batch_size;
        // Push output images into output vector
        for (int i = 0; i < current_batch_size; ++i)
        {
            output_img_uint8 = (uint8_t *)malloc(outputImageSize * sizeof(uint8_t));
            float* output_image_ptr = (float *)malloc(outputImageSize * sizeof(float));
            memcpy(output_image_ptr, output_imgs + i * outputImageSize, outputImageSize * sizeof(float));
            normalizeFloatImage(output_image_ptr, output_img_uint8, width, height);
            imagesOutput.push_back(output_img_uint8);
            free(output_image_ptr);
        }
        printf("Batch %d completed\n", batch_index + 1);
    }

    // Write output
    printf("\nWriting %d outputs ... \n", imagesOutput.size());
    for (int i = 0; i < imagesOutput.size(); ++i)
    {
        //std::string outputPath2 = std::string(output_folder) + "/original_" + std::to_string(i) + ".jpg";
        //stbi_write_jpg(outputPath2.c_str(), width, height, depth, images[i], 100); // original

        std::string outputPath = std::string(output_folder) + "/output_" + std::to_string(i) + ".jpg";
        stbi_write_jpg(outputPath.c_str(), width, height, 1, imagesOutput[i], 100); // convolution
        //printf("%d - Path : %s \n", i, outputPath.c_str());
    }
    printf("----------------------------------------------------------------------------------\n");
    // Free allocated memory
    for (int i = 0; i < images.size(); ++i)
    {
        stbi_image_free(images[i]);
        stbi_image_free(imagesOutput[i]);
    }

    free(output_imgs);
}

void launch_convolution_kernel(std::vector<uint8_t*> images, int width, int height, int depth, float *output_imgs, float *mask, int maskWidth, int batch_size, int batch_index, int current_batch_size)
{
    float *d_mask, *d_output;
    uint8_t *d_input;
    int imageSize = width * height * depth;
    int outputImageSize = width * height;

    cudaMalloc((void **)&d_input, imageSize * batch_size * sizeof(uint8_t));
    cudaMalloc((void **)&d_output, outputImageSize * batch_size * sizeof(float));
    cudaMalloc((void **)&d_mask, maskWidth * maskWidth * sizeof(float));

    uint8_t *input_imgs = (uint8_t *)malloc(batch_size * imageSize * sizeof(uint8_t));
    // Copy input images data from host to device
    for (int i = 0; i < current_batch_size; ++i) {
      memcpy(input_imgs + i * imageSize, images[i + batch_index * batch_size], imageSize * sizeof(uint8_t));
    }
    cudaMemcpy(d_input, input_imgs, batch_size * imageSize * sizeof(uint8_t), cudaMemcpyHostToDevice);
    cudaMemcpy(d_mask, mask, maskWidth * maskWidth * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(c_mask, mask, maskWidth * maskWidth * sizeof(float));

    // Define grid and block dimensions
    dim3 threadsPerBlock(16, 16, 1);
    dim3 numBlocks((width + threadsPerBlock.x - 1) / threadsPerBlock.x,
                  (height + threadsPerBlock.y - 1) / threadsPerBlock.y,
                   current_batch_size);
    // Launch kernel
    convolution3D<<<numBlocks, threadsPerBlock>>>(d_input, width, height, depth, d_output, d_mask, maskWidth);
    cudaDeviceSynchronize();

    cudaMemcpy(output_imgs, d_output, batch_size * outputImageSize * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_mask);
}

// Define a struct to hold image data along with its filename
struct ImageData {
    std::string filename;
    uint8_t *data;
};

// Comparator function to sort ImageData based on filenames
bool compare_filenames(const ImageData &a, const ImageData &b) {
    return a.filename < b.filename;
}
void load_images(std::vector<uint8_t*>& images, const char *input_folder, int batch_size, int &width, int &height, int &depth)
{
  std::vector<ImageData> images_ds;
    DIR *dir;
    struct dirent *ent;
    if ((dir = opendir(input_folder)) != NULL) {
        while ((ent = readdir(dir)) != NULL) {
            if (ent->d_type == DT_REG) { // Check if it's a regular file
                char inputPath[512];
                snprintf(inputPath, sizeof(inputPath), "%s/%s", input_folder, ent->d_name);

                uint8_t *inputImage = stbi_load(inputPath, &width, &height, &depth, 0);
                if (!inputImage) {
                    printf("Failed to load image: %s\n", inputPath);
                    continue; // Skip to the next image
                }

                // Store the filename and image data into the vector
                images_ds.push_back({ent->d_name, inputImage});
            }
        }
        closedir(dir);
    } else {
        perror("Failed to open directory");
        exit(EXIT_FAILURE);
    }

    // Sort the vector based on filenames
    std::sort(images_ds.begin(), images_ds.end(), compare_filenames);
    for (int i = 0; i < images_ds.size(); ++i)
    {
        images.push_back(images_ds[i].data);
    }
}


float *createMask(float** maskVal, int maskWidth) {

    // Allocate memory for the mask
    float *mask = new float[maskWidth * maskWidth];

    // Fill the mask with the Sobel filter values
    for (int i = 0; i < maskWidth; ++i) {
        for (int j = 0; j < maskWidth; ++j) {
            mask[i * maskWidth + j] = maskVal[i][j];
        }
    }
    return mask;
}

// Function to normalize float image to uint8_t
void normalizeFloatImage(float *img, uint8_t *output_img, int width, int height)
{
   float min_pixel = FLT_MAX;
   float max_pixel = -FLT_MAX;

   // Find min and max pixel values
   for (int i = 0; i < width * height; ++i)
   {
      if (img[i] < min_pixel)
      {
         min_pixel = img[i];
      }
      if (img[i] > max_pixel)
      {
         max_pixel = img[i];
      }
   }

   // Normalize and convert pixel values to uint8_t
   for (int i = 0; i < width * height; ++i)
   {
      float normalized_pixel = ((img[i] - min_pixel) / (max_pixel - min_pixel)) * 255.0f;
      output_img[i] = (uint8_t)normalized_pixel;
   }
}

Overwriting K1_1_17_2_14.cu


In [ ]:
!nvcc -w K1_1_17_2_14.cu -o kernel1.out
# !./kernel1.out Input Output 4 mask.txt

In [ ]:
!nvprof ./kernel1.out Input Output 4 mask.txt

Started processing 4 images with dimensions : (1280, 720, 3) 
==18050== NVPROF is profiling process 18050, command: ./kernel1.out Input Output 4 mask.txt
Batch 1 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 2 completed

Writing 8 outputs ... 
----------------------------------------------------------------------------------
==18050== Profiling application: ./kernel1.out Input Output 4 mask.txt
==18050== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   63.88%  16.222ms         2  8.1111ms  3.4242ms  12.798ms  [CUDA memcpy DtoH]
                   19.34%  4.9103ms         6  818.38us     672ns  2.5721ms  [CUDA memcpy HtoD]
                   16.79%  4.2626ms         2  2.1313ms  2.1311ms  2.1314ms  convolution3D(unsigned char const *, int, int, int, float*, float*, int)
      API calls:   87.13%  211.83ms         6  35.305ms  118.49us  211.02ms  cudaMalloc
                    9.44%  22.

# kernel 2: tiling where each block matches the input tile size.

In [ ]:
%%writefile "K2_1_17_2_14.cu"
// nvcc -w K2_1_17_2_14.cu -o kernel2.out
// ./kernel2.out Input Output 4 mask.txt
#include <algorithm>
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <stdlib.h>
#include <string.h>
#include <float.h>
#include "dirent.h"

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"

// Define tile dimensions
#define O_TILE_WIDTH 16


#define MAX_MASK_WIDTH 25
__constant__ float c_mask[MAX_MASK_WIDTH];

// Kernel function for 3D convolution with tiling
__global__ void convolution3D(const uint8_t *input, int width, int height, int depth, float *output, float * mask, int maskWidth)
{
    // Define shared memory size dynamically
    extern __shared__ float inputTile[];

    // Thread indices
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Output indices
    int col_o = blockIdx.x * O_TILE_WIDTH + tx;
    int row_o = blockIdx.y * O_TILE_WIDTH + ty;

    // Input indices
    int col_i = col_o - maskWidth / 2;
    int row_i = row_o - maskWidth / 2;

    int batch_index = blockIdx.z;
    int image_index_in_batch = batch_index * width * height * depth;

    float pixelValue = 0.0;
    for (int channel = 0; channel < depth; channel++)
    {
        // Load input tile into shared memory
        if (col_i >= 0 && col_i < width && row_i >= 0 && row_i < height)
            inputTile[ty * (O_TILE_WIDTH + maskWidth - 1) + tx] = static_cast<float>(input[(row_i * width + col_i) * depth + channel + image_index_in_batch]) / 255.0f;
        else
            inputTile[ty * (O_TILE_WIDTH + maskWidth - 1) + tx] = 0.0f;

        __syncthreads();

        // Compute convolution
        if (tx < O_TILE_WIDTH && ty < O_TILE_WIDTH)
        {
            for (int j = 0; j < maskWidth; j++)
                for (int k = 0; k < maskWidth; k++)
                    pixelValue += inputTile[(ty + j) * (O_TILE_WIDTH + maskWidth - 1) + (tx + k)] * c_mask[j * maskWidth + k];

        }

        __syncthreads();
    }
    if (tx < O_TILE_WIDTH && ty < O_TILE_WIDTH)
    {
        // Write output
        if (col_o < width && row_o < height)
        {
            int outIndex =  (row_o * width + col_o) + batch_index * width * height;
            output[outIndex] = pixelValue;
        }
    }
}



void process_batch(const char *input_folder, const char *output_folder, float *mask, int maskWidth, int batch_size);
void launch_convolution_kernel(std::vector<uint8_t*> images, int width, int height, int depth, float *output_imgs, float *mask, int maskWidth, int batch_size, int batch_index, int current_batch_size);
void load_images(std::vector<uint8_t*>& images, const char *input_folder, int batch_size, int &width, int &height, int &depth) ;
float *createMask(float** maskVal, int maskWidth);
void normalizeFloatImage(float *img, uint8_t *output_img, int width, int height);

int main(int argc, char **argv)
{
    if (argc < 5)
    {
        std::cerr << "Usage: " << argv[0] << " <input_folder> <output_folder> <batch_size> <mask_file>" << std::endl;
        return 1;
    }


    // Load input images from folder
    const char* inputFolder = argv[1];
    const char* outputFolder = argv[2];
    int batch_size = std::atoi(argv[3]);
    const char* maskfilePath = argv[4];
    std::ifstream inFile(maskfilePath); // Open the mask file
    if (!inFile) {
        std::cerr << "Unable to open file mask.txt" << std::endl;
        return 1;
    }

    int maskWidth;
    inFile >> maskWidth; // Read mask width from the first line

    float** maskVal = new float*[maskWidth]; // Allocate memory for rows
    for (int i = 0; i < maskWidth; ++i) {
        maskVal[i] = new float[maskWidth]; // Allocate memory for columns
    }


    // Read mask values from the file
    for (int i = 0; i < maskWidth; ++i) {
        for (int j = 0; j < maskWidth; ++j) {
            std::string maskValue;
            if (!(inFile >> maskValue)) {
                std::cerr << "Error reading mask value" << std::endl;
                return 1;
            }

            // Parse the fraction if it contains '/'
            size_t pos = maskValue.find('/');
            if (pos != std::string::npos) {
                int numerator = std::atoi(maskValue.substr(0, pos).c_str());
                int denominator = std::atoi(maskValue.substr(pos + 1).c_str());
                if (denominator == 0) {
                    std::cerr << "Invalid denominator in mask value" << std::endl;
                    return 1;
                }
                maskVal[i][j] = static_cast<double>(numerator) / denominator;
            } else {
                // If there's no '/', parse the value directly
                maskVal[i][j] = std::atof(maskValue.c_str());
            }
        }
    }
    inFile.close();

    // Create mask using the provided function
    float *mask = createMask(maskVal, maskWidth);

    process_batch(inputFolder, outputFolder, mask, maskWidth, batch_size);

    delete[] mask;
    return 0;
}

void process_batch(const char *input_folder, const char *output_folder, float *mask, int maskWidth, int batch_size)
{
    // Load images from input folder
    int width, height, depth;
    std::vector<uint8_t*> images;
    std::vector<uint8_t*> imagesOutput;
    load_images(images, input_folder, batch_size, width, height, depth);

    int imageSize = width * height * depth;
    int outputImageSize = width * height;
    int number_of_images = images.size();
    int num_iterations = (number_of_images + batch_size - 1) / batch_size;
    int remaining_images = number_of_images;
    uint8_t* output_img_uint8;
    float *output_imgs = (float *)malloc(batch_size * outputImageSize * sizeof(float));

    for (int batch_index = 0; batch_index < num_iterations; ++batch_index)
    {
        // Start convolution batch processing
        int current_batch_size = min(batch_size, remaining_images);
        printf("Started processing %d images with dimensions : (%d, %d, %d) \n", current_batch_size, width, height, depth);
        launch_convolution_kernel(images, width, height, depth, output_imgs, mask, maskWidth, batch_size, batch_index, current_batch_size);
        remaining_images -= batch_size;
        // Push output images into output vector
        for (int i = 0; i < current_batch_size; ++i)
        {
            output_img_uint8 = (uint8_t *)malloc(outputImageSize * sizeof(uint8_t));
            float* output_image_ptr = (float *)malloc(outputImageSize * sizeof(float));
            memcpy(output_image_ptr, output_imgs + i * outputImageSize, outputImageSize * sizeof(float));
            normalizeFloatImage(output_image_ptr, output_img_uint8, width, height);
            imagesOutput.push_back(output_img_uint8);
            free(output_image_ptr);
        }
        printf("Batch %d completed\n", batch_index + 1);
    }

    // Write output
    printf("\nWriting %d outputs ... \n", imagesOutput.size());
    for (int i = 0; i < imagesOutput.size(); ++i)
    {
        //std::string outputPath2 = std::string(output_folder) + "/original_" + std::to_string(i) + ".jpg";
        //stbi_write_jpg(outputPath2.c_str(), width, height, depth, images[i], 100); // original

        std::string outputPath = std::string(output_folder) + "/output_" + std::to_string(i) + ".jpg";
        stbi_write_jpg(outputPath.c_str(), width, height, 1, imagesOutput[i], 100); // convolution
        //printf("%d - Path : %s \n", i, outputPath.c_str());
    }
    printf("----------------------------------------------------------------------------------\n");

    // Free allocated memory
    for (int i = 0; i < images.size(); ++i)
    {
        stbi_image_free(images[i]);
        stbi_image_free(imagesOutput[i]);
    }

    free(output_imgs);
}

void launch_convolution_kernel(std::vector<uint8_t*> images, int width, int height, int depth, float *output_imgs, float *mask, int maskWidth, int batch_size, int batch_index, int current_batch_size)
{
    float *d_mask, *d_output;
    uint8_t *d_input;
    int imageSize = width * height * depth;
    int outputImageSize = width * height;

    cudaMalloc((void **)&d_input, imageSize * batch_size * sizeof(uint8_t));
    cudaMalloc((void **)&d_output, outputImageSize * batch_size * sizeof(float));
    cudaMalloc((void **)&d_mask, maskWidth * maskWidth * sizeof(float));

    uint8_t *input_imgs = (uint8_t *)malloc(batch_size * imageSize * sizeof(uint8_t));
    // Copy input images data from host to device
    for (int i = 0; i < current_batch_size; ++i) {
      memcpy(input_imgs + i * imageSize, images[i + batch_index * batch_size], imageSize * sizeof(uint8_t));
    }
    cudaMemcpy(d_input, input_imgs, batch_size * imageSize * sizeof(uint8_t), cudaMemcpyHostToDevice);
    cudaMemcpy(d_mask, mask, maskWidth * maskWidth * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(c_mask, mask, maskWidth * maskWidth * sizeof(float));

    // Define grid and block dimensions
    dim3 threadsPerBlock(O_TILE_WIDTH + maskWidth - 1, O_TILE_WIDTH + maskWidth - 1, 1);
    dim3 numBlocks((width - 1) / O_TILE_WIDTH + 1,
                   (height - 1) / O_TILE_WIDTH + 1,
                    current_batch_size);
    int sharedMemorySize = sizeof(float) * (O_TILE_WIDTH + maskWidth - 1) * (O_TILE_WIDTH + maskWidth - 1);
    // Launch kernel
    convolution3D<<<numBlocks, threadsPerBlock, sharedMemorySize>>>(d_input, width, height, depth, d_output, d_mask, maskWidth);
    cudaDeviceSynchronize();

    cudaMemcpy(output_imgs, d_output, batch_size * outputImageSize * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_mask);
}

// Define a struct to hold image data along with its filename
struct ImageData {
    std::string filename;
    uint8_t *data;
};

// Comparator function to sort ImageData based on filenames
bool compare_filenames(const ImageData &a, const ImageData &b) {
    return a.filename < b.filename;
}
void load_images(std::vector<uint8_t*>& images, const char *input_folder, int batch_size, int &width, int &height, int &depth)
{
    std::vector<ImageData> images_ds;
    DIR *dir;
    struct dirent *ent;
    if ((dir = opendir(input_folder)) != NULL) {
        while ((ent = readdir(dir)) != NULL) {
            if (ent->d_type == DT_REG) { // Check if it's a regular file
                char inputPath[512];
                snprintf(inputPath, sizeof(inputPath), "%s/%s", input_folder, ent->d_name);

                uint8_t *inputImage = stbi_load(inputPath, &width, &height, &depth, 0);
                if (!inputImage) {
                    printf("Failed to load image: %s\n", inputPath);
                    continue; // Skip to the next image
                }

                // Store the filename and image data into the vector
                images_ds.push_back({ent->d_name, inputImage});
            }
        }
        closedir(dir);
    } else {
        perror("Failed to open directory");
        exit(EXIT_FAILURE);
    }

    // Sort the vector based on filenames
    std::sort(images_ds.begin(), images_ds.end(), compare_filenames);
    for (int i = 0; i < images_ds.size(); ++i)
    {
        images.push_back(images_ds[i].data);
    }
}


float *createMask(float** maskVal, int maskWidth) {

    // Allocate memory for the mask
    float *mask = new float[maskWidth * maskWidth];

    // Fill the mask with the Sobel filter values
    for (int i = 0; i < maskWidth; ++i) {
        for (int j = 0; j < maskWidth; ++j) {
            mask[i * maskWidth + j] = maskVal[i][j];
        }
    }
    return mask;
}

// Function to normalize float image to uint8_t
void normalizeFloatImage(float *img, uint8_t *output_img, int width, int height)
{
   float min_pixel = FLT_MAX;
   float max_pixel = -FLT_MAX;

   // Find min and max pixel values
   for (int i = 0; i < width * height; ++i)
   {
      if (img[i] < min_pixel)
      {
         min_pixel = img[i];
      }
      if (img[i] > max_pixel)
      {
         max_pixel = img[i];
      }
   }

   // Normalize and convert pixel values to uint8_t
   for (int i = 0; i < width * height; ++i)
   {
      float normalized_pixel = ((img[i] - min_pixel) / (max_pixel - min_pixel)) * 255.0f;
      output_img[i] = (uint8_t)normalized_pixel;
   }
}

Overwriting K2_1_17_2_14.cu


In [ ]:
!nvcc -w K2_1_17_2_14.cu -o kernel2.out

In [ ]:
!nvprof ./kernel2.out Input Output2 4 mask.txt

Started processing 4 images with dimensions : (1280, 720, 3) 
==18011== NVPROF is profiling process 18011, command: ./kernel2.out Input Output2 4 mask.txt
Batch 1 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 2 completed

Writing 8 outputs ... 
----------------------------------------------------------------------------------
==18011== Profiling application: ./kernel2.out Input Output2 4 mask.txt
==18011== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.11%  13.418ms         2  6.7088ms  3.2001ms  10.217ms  [CUDA memcpy DtoH]
                   21.19%  4.7291ms         2  2.3646ms  2.3525ms  2.3766ms  convolution3D(unsigned char const *, int, int, int, float*, float*, int)
                   18.71%  4.1759ms         6  695.99us     672ns  2.0927ms  [CUDA memcpy HtoD]
      API calls:   86.25%  171.92ms         6  28.654ms  76.087us  171.39ms  cudaMalloc
                    9.64%  1

# kernel 3: tiling where each block matches the output tile size.

In [ ]:
%%writefile "K3_1_17_2_14.cu"
// nvcc -w K3_1_17_2_14.cu -o kernel3.out
// ./kernel3.out Input Output 4 mask.txt
#include <algorithm>
#include <iostream>
#include <fstream>
#include <sstream>
#include <vector>
#include <stdlib.h>
#include <string.h>
#include <float.h>
#include "dirent.h"

#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"
#define STB_IMAGE_WRITE_IMPLEMENTATION
#include "stb_image_write.h"


//#define MAX_MASK_SIZE 25  // Maximum size of the mask
//__constant__ float d_mask[MAX_MASK_SIZE];

// Define tile dimensions
#define O_TILE_WIDTH 32



#define MAX_MASK_WIDTH 25
__constant__ float c_mask[MAX_MASK_WIDTH];

// Kernel function for 3D convolution with tiling
__global__ void convolution3D(const uint8_t *input, int width, int height, int depth, float *output, const float *__restrict__ mask, int maskWidth)
{
    // Shared memory for input tile
    extern __shared__ float inputTile[];

    // Thread indices
    int tx = threadIdx.x;
    int ty = threadIdx.y;

    // Output indices
    int col_o = blockIdx.x * blockDim.x + tx;
    int row_o = blockIdx.y * blockDim.y + ty;

    // Input indices
    int col_i = col_o - maskWidth / 2;
    int row_i = row_o - maskWidth / 2;

    int batch_index = blockIdx.z;
    int image_index_in_batch = batch_index * width * height * depth;

    float pixelValue = 0.0f;
    for (int channel = 0; channel < depth; channel++)
    {
        // Load input tile into shared memory
        if (col_i >= 0 && col_i < width && row_i >= 0 && row_i < height)
            inputTile[ty * (O_TILE_WIDTH + maskWidth - 1) + tx] = static_cast<float>(input[(row_i * width + col_i) * depth + channel + image_index_in_batch]) / 255.0f;
        else
            inputTile[ty * (O_TILE_WIDTH + maskWidth - 1) + tx] = 0.0f;

        // Load additional input elements into shared memory
        if (tx < maskWidth - 1)
        {
            int col_i2 = col_o + blockDim.x - maskWidth / 2;
            if (col_i2 < width && row_i < height)
                inputTile[ty * (O_TILE_WIDTH + maskWidth - 1) + tx + blockDim.x] = static_cast<float>(input[(row_i * width + col_i2) * depth + channel + image_index_in_batch]) / 255.0f;
            else
                inputTile[ty * (O_TILE_WIDTH + maskWidth - 1) + tx + blockDim.x] = 0.0f;
        }

        if (ty < maskWidth - 1)
        {
            int row_i2 = row_o + blockDim.y - maskWidth / 2;
            if (col_i < width && row_i2 < height)
                inputTile[(ty + blockDim.y) * (O_TILE_WIDTH + maskWidth - 1) + tx] = static_cast<float>(input[(row_i2 * width + col_i) * depth + channel + image_index_in_batch]) / 255.0f;
            else
                inputTile[(ty + blockDim.y) * (O_TILE_WIDTH + maskWidth - 1) + tx] = 0.0f;
        }

        // Load corners of input tile into shared memory
        if (tx < maskWidth - 1 && ty < maskWidth - 1)
        {
            int col_i2 = col_o + blockDim.x - maskWidth / 2;
            int row_i2 = row_o + blockDim.y - maskWidth / 2;
            if (col_i2 < width && row_i2 < height)
                inputTile[(ty + blockDim.y) * (O_TILE_WIDTH + maskWidth - 1) + tx + blockDim.x] = static_cast<float>(input[(row_i2 * width + col_i2) * depth + channel + image_index_in_batch]) / 255.0f;
            else
                inputTile[(ty + blockDim.y) * (O_TILE_WIDTH + maskWidth - 1) + tx + blockDim.x] = 0.0f;
        }
        __syncthreads();

        // Compute convolution
        if (ty < blockDim.y && tx <  blockDim.x)
        {
            for (int j = 0; j < maskWidth; j++)
                for (int k = 0; k < maskWidth; k++)
                    pixelValue += inputTile[(ty + j) * (O_TILE_WIDTH + maskWidth - 1) + (tx + k)] * c_mask[j * maskWidth + k];
        }
    }
    // Write output
    if (col_o < width && row_o < height)
    {
        int outIndex =  (row_o * width + col_o) + batch_index * width * height;
        output[outIndex] = pixelValue;
    }
}



void process_batch(const char *input_folder, const char *output_folder, float *mask, int maskWidth, int batch_size);
void launch_convolution_kernel(std::vector<uint8_t*> images, int width, int height, int depth, float *output_imgs, float *mask, int maskWidth, int batch_size, int batch_index, int current_batch_size);
void load_images(std::vector<uint8_t*>& images, const char *input_folder, int batch_size, int &width, int &height, int &depth) ;
float *createMask(float** maskVal, int maskWidth);
void normalizeFloatImage(float *img, uint8_t *output_img, int width, int height);

int main(int argc, char **argv)
{
    if (argc < 5)
    {
        std::cerr << "Usage: " << argv[0] << " <input_folder> <output_folder> <batch_size> <mask_file>" << std::endl;
        return 1;
    }


    // Load input images from folder
    const char* inputFolder = argv[1];
    const char* outputFolder = argv[2];
    int batch_size = std::atoi(argv[3]);
    const char* maskfilePath = argv[4];
    std::ifstream inFile(maskfilePath); // Open the mask file
    if (!inFile) {
        std::cerr << "Unable to open file mask.txt" << std::endl;
        return 1;
    }

    int maskWidth;
    inFile >> maskWidth; // Read mask width from the first line

    float** maskVal = new float*[maskWidth]; // Allocate memory for rows
    for (int i = 0; i < maskWidth; ++i) {
        maskVal[i] = new float[maskWidth]; // Allocate memory for columns
    }


    // Read mask values from the file
    for (int i = 0; i < maskWidth; ++i) {
        for (int j = 0; j < maskWidth; ++j) {
            std::string maskValue;
            if (!(inFile >> maskValue)) {
                std::cerr << "Error reading mask value" << std::endl;
                return 1;
            }

            // Parse the fraction if it contains '/'
            size_t pos = maskValue.find('/');
            if (pos != std::string::npos) {
                int numerator = std::atoi(maskValue.substr(0, pos).c_str());
                int denominator = std::atoi(maskValue.substr(pos + 1).c_str());
                if (denominator == 0) {
                    std::cerr << "Invalid denominator in mask value" << std::endl;
                    return 1;
                }
                maskVal[i][j] = static_cast<double>(numerator) / denominator;
            } else {
                // If there's no '/', parse the value directly
                maskVal[i][j] = std::atof(maskValue.c_str());
            }
        }
    }
    inFile.close();


    // Create mask using the provided function
    float *mask = createMask(maskVal, maskWidth);

    process_batch(inputFolder, outputFolder, mask, maskWidth, batch_size);

    delete[] mask;
    return 0;
}

void process_batch(const char *input_folder, const char *output_folder, float *mask, int maskWidth, int batch_size)
{
    // Load images from input folder
    int width, height, depth;
    std::vector<uint8_t*> images;
    std::vector<uint8_t*> imagesOutput;
    load_images(images, input_folder, batch_size, width, height, depth);

    int imageSize = width * height * depth;
    int outputImageSize = width * height;
    int number_of_images = images.size();
    int num_iterations = (number_of_images + batch_size - 1) / batch_size;
    int remaining_images = number_of_images;
    uint8_t* output_img_uint8;
    float *output_imgs = (float *)malloc(batch_size * outputImageSize * sizeof(float));

    for (int batch_index = 0; batch_index < num_iterations; ++batch_index)
    {
        // Start convolution batch processing
        int current_batch_size = min(batch_size, remaining_images);
        printf("Started processing %d images with dimensions : (%d, %d, %d) \n", current_batch_size, width, height, depth);
        launch_convolution_kernel(images, width, height, depth, output_imgs, mask, maskWidth, batch_size, batch_index, current_batch_size);
        remaining_images -= batch_size;
        // Push output images into output vector
        for (int i = 0; i < current_batch_size; ++i)
        {
            output_img_uint8 = (uint8_t *)malloc(outputImageSize * sizeof(uint8_t));
            float* output_image_ptr = (float *)malloc(outputImageSize * sizeof(float));
            memcpy(output_image_ptr, output_imgs + i * outputImageSize, outputImageSize * sizeof(float));
            normalizeFloatImage(output_image_ptr, output_img_uint8, width, height);
            imagesOutput.push_back(output_img_uint8);
            free(output_image_ptr);
        }
        printf("Batch %d completed\n", batch_index + 1);
    }

    // Write output
    printf("\nWriting %d outputs ... \n", imagesOutput.size());
    for (int i = 0; i < imagesOutput.size(); ++i)
    {
        //std::string outputPath2 = std::string(output_folder) + "/original_" + std::to_string(i) + ".jpg";
        //stbi_write_jpg(outputPath2.c_str(), width, height, depth, images[i], 100); // original

        std::string outputPath = std::string(output_folder) + "/output_" + std::to_string(i) + ".jpg";
        stbi_write_jpg(outputPath.c_str(), width, height, 1, imagesOutput[i], 100); // convolution
        //printf("%d - Path : %s \n", i, outputPath.c_str());
    }
    printf("----------------------------------------------------------------------------------\n");

    // Free allocated memory
    for (int i = 0; i < images.size(); ++i)
    {
        stbi_image_free(images[i]);
        stbi_image_free(imagesOutput[i]);
    }

    free(output_imgs);
}

void launch_convolution_kernel(std::vector<uint8_t*> images, int width, int height, int depth, float *output_imgs, float *mask, int maskWidth, int batch_size, int batch_index, int current_batch_size)
{
    float *d_mask, *d_output;
    uint8_t *d_input;
    int imageSize = width * height * depth;
    int outputImageSize = width * height;

    cudaMalloc((void **)&d_input, imageSize * batch_size * sizeof(uint8_t));
    cudaMalloc((void **)&d_output, outputImageSize * batch_size * sizeof(float));
    cudaMalloc((void **)&d_mask, maskWidth * maskWidth * sizeof(float));

    uint8_t *input_imgs = (uint8_t *)malloc(batch_size * imageSize * sizeof(uint8_t));
    // Copy input images data from host to device
    for (int i = 0; i < current_batch_size; ++i) {
      memcpy(input_imgs + i * imageSize, images[i + batch_index * batch_size], imageSize * sizeof(uint8_t));
    }
    cudaMemcpy(d_input, input_imgs, batch_size * imageSize * sizeof(uint8_t), cudaMemcpyHostToDevice);
    cudaMemcpy(d_mask, mask, maskWidth * maskWidth * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpyToSymbol(c_mask, mask, maskWidth * maskWidth * sizeof(float));

    // Define grid and block dimensions
    dim3 threadsPerBlock(O_TILE_WIDTH, O_TILE_WIDTH, 1);
    dim3 numBlocks((width - 1) / O_TILE_WIDTH + 1, (height - 1) / O_TILE_WIDTH + 1, current_batch_size);

    int sharedMemorySize = sizeof(float) * (O_TILE_WIDTH + maskWidth - 1) * (O_TILE_WIDTH + maskWidth - 1);
    // Launch kernel
    convolution3D<<<numBlocks, threadsPerBlock, sharedMemorySize>>>(d_input, width, height, depth, d_output, d_mask, maskWidth);
    cudaDeviceSynchronize();

    cudaMemcpy(output_imgs, d_output, batch_size * outputImageSize * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_mask);
}

// Define a struct to hold image data along with its filename
struct ImageData {
    std::string filename;
    uint8_t *data;
};

// Comparator function to sort ImageData based on filenames
bool compare_filenames(const ImageData &a, const ImageData &b) {
    return a.filename < b.filename;
}
void load_images(std::vector<uint8_t*>& images, const char *input_folder, int batch_size, int &width, int &height, int &depth)
{
    std::vector<ImageData> images_ds;
    DIR *dir;
    struct dirent *ent;
    if ((dir = opendir(input_folder)) != NULL) {
        while ((ent = readdir(dir)) != NULL) {
            if (ent->d_type == DT_REG) { // Check if it's a regular file
                char inputPath[512];
                snprintf(inputPath, sizeof(inputPath), "%s/%s", input_folder, ent->d_name);

                uint8_t *inputImage = stbi_load(inputPath, &width, &height, &depth, 0);
                if (!inputImage) {
                    printf("Failed to load image: %s\n", inputPath);
                    continue; // Skip to the next image
                }

                // Store the filename and image data into the vector
                images_ds.push_back({ent->d_name, inputImage});
            }
        }
        closedir(dir);
    } else {
        perror("Failed to open directory");
        exit(EXIT_FAILURE);
    }

    // Sort the vector based on filenames
    std::sort(images_ds.begin(), images_ds.end(), compare_filenames);
    for (int i = 0; i < images_ds.size(); ++i)
    {
        images.push_back(images_ds[i].data);
    }
}


float *createMask(float** maskVal, int maskWidth) {

    // Allocate memory for the mask
    float *mask = new float[maskWidth * maskWidth];

    // Fill the mask with the Sobel filter values
    for (int i = 0; i < maskWidth; ++i) {
        for (int j = 0; j < maskWidth; ++j) {
            mask[i * maskWidth + j] = maskVal[i][j];
        }
    }
    return mask;
}

// Function to normalize float image to uint8_t
void normalizeFloatImage(float *img, uint8_t *output_img, int width, int height)
{
   float min_pixel = FLT_MAX;
   float max_pixel = -FLT_MAX;

   // Find min and max pixel values
   for (int i = 0; i < width * height; ++i)
   {
      if (img[i] < min_pixel)
      {
         min_pixel = img[i];
      }
      if (img[i] > max_pixel)
      {
         max_pixel = img[i];
      }
   }

   // Normalize and convert pixel values to uint8_t
   for (int i = 0; i < width * height; ++i)
   {
      float normalized_pixel = ((img[i] - min_pixel) / (max_pixel - min_pixel)) * 255.0f;
      output_img[i] = (uint8_t)normalized_pixel;
   }
}

Overwriting K3_1_17_2_14.cu


In [ ]:
!nvcc -w K3_1_17_2_14.cu -o kernel3.out

In [ ]:
!nvprof ./kernel3.out Input32 Output2 4 mask.txt

Started processing 4 images with dimensions : (1280, 720, 3) 
==16315== NVPROF is profiling process 16315, command: ./kernel3.out Input32 Output2 4 mask.txt
Batch 1 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 2 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 3 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 4 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 5 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 6 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 7 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 8 completed

Writing 32 outputs ... 
----------------------------------------------------------------------------------
==16315== Profiling application: ./kernel3.out Input32 Output2 4 mask.txt
==16315== Profiling result:
            Type  Time(%)      Time     Calls       Avg

# Pytorch

In [ ]:
# !./kernel1.out Input Output 4 mask.txt
# !./kernel2.out Input Output2 4 mask.txt
# !./kernel3.out Input Output3 4 mask.txt

In [ ]:
# %%writefile "B_1_17_2_14.py"
# python B_1_17_2_14.py Input PytorchOutput 4 mask.txt
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import time
import sys


def load_kernel(kernel_path):
    # Read kernel from file
    kernel_file = "mask.txt"
    kernel3d = []
    with open(kernel_file, "r") as f:
        lines = f.readlines()
        kernel_size = int(lines[0].strip())  # Read kernel size
        kernel_values = []
        for line in lines[1:]:
            values = line.split()
            parsed_values = []
            for value in values:
                if '/' in value:
                    numerator, denominator = map(int, value.split('/'))
                    parsed_values.append(numerator / denominator)
                else:
                    parsed_values.append(float(value))
            kernel_values.append(parsed_values)

        for i in range(3):  # each channel
            kernel3d.append(kernel_values)
    # Define convolution kernel
    kernel = torch.tensor(kernel3d)
    return kernel, kernel_size


def load_input_images(input_folder):
  input_files = sorted(os.listdir(input_folder))
  input_images = []
  for input_file in input_files:
    input_path = os.path.join(input_folder, input_file)
    input_images.append(Image.open(input_path))
  return input_images


def pytorch_convolution(input_images, kernel, batch_size=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using", device)
    transform = transforms.Compose([transforms.ToTensor()])
    pytorch_outputs = []

    timings = {'total_execution_time_ms': 0, 'batch_times_ms': []}

    for i in range(0, len(input_images), batch_size):

        batch_images = input_images[i:i+batch_size]
        batch_tensors = []
        for image in batch_images:
            image_tensor = transform(image).unsqueeze(0).to(device)
            batch_tensors.append(image_tensor)

        batch_tensor = torch.cat(batch_tensors, dim=0)

        # Apply padding
        padding = kernel.size(2) // 2  # Assuming square kernel
        batch_tensor_padded = F.pad(batch_tensor, (padding, padding, padding, padding), mode='constant', value=0)

        # Apply convolution
        batch_start_time = time.time()
        output = F.conv3d(batch_tensor_padded.unsqueeze(1), kernel.unsqueeze(0).unsqueeze(0).to(device))
        batch_end_time = time.time()

        # Convert output tensor to numpy array and normalize
        output_images = output.squeeze(1).cpu().numpy().transpose(0, 2, 3, 1)

        for out in output_images:
            pytorch_outputs.append(out)

        batch_time_ms = (batch_end_time - batch_start_time) * 1000
        timings['batch_times_ms'].append(batch_time_ms)

    timings['total_execution_time_ms'] = sum(timings['batch_times_ms'])

    return pytorch_outputs, timings

def save_output(pytorch_outputs, output_path):
  for i in range(len(pytorch_outputs)):
    # Save PyTorch output image as JPEG
    output_image_tosave = (pytorch_outputs[i] - pytorch_outputs[i].min()) / (pytorch_outputs[i].max() - pytorch_outputs[i].min()) * 255
    output_image_tosave = output_image_tosave.astype('uint8')
    if output_image_tosave.ndim == 3 and output_image_tosave.shape[2] == 1:
        output_image_tosave = output_image_tosave.squeeze(2)  # Remove the third dimension if it's 1
    output_image_tosave = Image.fromarray(output_image_tosave)
    output_image_tosave.save(os.path.join(output_path, f"pytorch_output_{i}.jpg"))



def profile(timings):
  print("Total execution time of F.conv3d (ms) :", timings['total_execution_time_ms'])
  for i, batch_time in enumerate(timings['batch_times_ms']):
    print(f"\t Batch {i+1} execution time (ms) : {batch_time}")

def main():
  # if len(sys.argv) != 5:
  #     print("Usage: python B_1_17_2_14.py <input_folder> <output_folder> <batch_size> <mask_file>")
  #     return

  input_folder = "Input" # sys.argv[1]
  pytorch_folder = "PytorchOutput" # sys.argv[2]
  batch_size = 4 # int(sys.argv[3])
  mask_file_path = "mask.txt" # sys.argv[4]

  # Define convolution kernel
  kernel, kernel_size = load_kernel(mask_file_path)

  # Define batch size
  print("Loading Input Images...")
  input_images = load_input_images(input_folder)


  print("Processing...")
  pytorch_outputs, timings = pytorch_convolution(input_images, kernel, batch_size)

  print("Writing Output...")
  save_output(pytorch_outputs, pytorch_folder)

  profile(timings)


if __name__ == "__main__":
    main()

# TO RUN:
# python B_1_17_2_14.py Input PytorchOutput 4 mask.txt

Loading Input Images...
Processing...
Using cuda
Writing Output...
Total execution time of F.conv3d (ms) : 8.32223892211914
	 Batch 1 execution time (ms) : 1.119852066040039
	 Batch 2 execution time (ms) : 1.0330677032470703
	 Batch 3 execution time (ms) : 1.0290145874023438
	 Batch 4 execution time (ms) : 1.0187625885009766
	 Batch 5 execution time (ms) : 1.0340213775634766
	 Batch 6 execution time (ms) : 1.0268688201904297
	 Batch 7 execution time (ms) : 1.0285377502441406
	 Batch 8 execution time (ms) : 1.032114028930664


In [ ]:
!python B_1_17_2_14.py Input PytorchOutput 4 mask.txt

Loading Input Images...
Processing...
Using cuda
Writing Output...
Total execution time of F.conv3d (ms) : 152.587890625
	 Batch 1 execution time (ms) : 151.58796310424805
	 Batch 2 execution time (ms) : 0.9999275207519531


In [ ]:
# from PIL import Image
# import os
# import matplotlib.pyplot as plt


# def load_input_images(input_folder):
#   input_files = sorted(os.listdir("Input"))
#   input_images = []
#   for input_file in input_files:
#     input_path = os.path.join(input_folder, input_file)
#     input_images.append(Image.open(input_path))
#   return input_images

# def load_output_image(output_folder):
#   output_images = []
#   output_files = sorted(os.listdir(output_folder))
#   for output_file in output_files:
#     output_path = os.path.join(output_folder, output_file)
#     cuda_image = Image.open(output_path)
#     output_images.append(cuda_image)
#   return output_images

# def load_cuda_output_images(output_folders):
#   outputs = []
#   for folder in output_folders:
#     outputs.append(load_output_image(folder))
#   return outputs

# def display(input_images, pytorch_outputs, cuda_output_images):
#   fig, axes = plt.subplots(len(input_images), 5, figsize=(25, len(input_images)*3))
#   for i in range(len(pytorch_outputs)):
#     cuda_image_without_tilling = cuda_output_images[0][i]
#     cuda_image_with_tilling = cuda_output_images[1][i]
#     cuda_image_with_tilling_2 = cuda_output_images[2][i]
#     # Display input and output images
#     axes[i][0].imshow(input_images[i])
#     axes[i][0].set_title("Input Image")
#     axes[i][0].axis("off")

#     axes[i][1].imshow(pytorch_outputs[i])
#     axes[i][1].set_title("Pytorch Image")
#     axes[i][1].axis("off")


#     axes[i][2].imshow(cuda_image_without_tilling)
#     axes[i][2].set_title("CUDA Without Tilling")
#     axes[i][2].axis("off")


#     axes[i][3].imshow(cuda_image_with_tilling)
#     axes[i][3].set_title("CUDA With input Tilling")
#     axes[i][3].axis("off")


#     axes[i][4].imshow(cuda_image_with_tilling_2)
#     axes[i][4].set_title("CUDA With output Tilling")
#     axes[i][4].axis("off")

#   plt.show()

# output_folders = ["Output" , "Output2" , "Output3"]
# pytorch_output_folder = "PytorchOutput"
# input_images = load_input_images("Input")
# cuda_output_images = load_cuda_output_images(output_folders)
# pytorch_outputs = load_output_image(pytorch_output_folder)
# display(input_images, pytorch_outputs, cuda_output_images)

# Profiling performance

### Kernel 1

In [ ]:
!nvprof ./kernel1.out Input Output 4 mask.txt

Started processing 4 images with dimensions : (1280, 720, 3) 
==1266== NVPROF is profiling process 1266, command: ./kernel1.out Input Output 4 mask.txt
Batch 1 completed
Started processing 4 images with dimensions : (1280, 720, 3) 
Batch 2 completed

Writing 8 outputs ... 
----------------------------------------------------------------------------------
==1266== Profiling application: ./kernel1.out Input Output 4 mask.txt
==1266== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   67.57%  13.275ms         2  6.6374ms  3.2185ms  10.056ms  [CUDA memcpy DtoH]
                   22.02%  4.3272ms         4  1.0818ms  1.1840us  2.1696ms  [CUDA memcpy HtoD]
                   10.41%  2.0453ms         2  1.0227ms  1.0226ms  1.0227ms  convolution3D(unsigned char const *, int, int, int, float*, float const *, int)
      API calls:   88.15%  183.81ms         6  30.635ms  83.917us  183.08ms  cudaMalloc
                    9.21%  

### Kernel 2

In [ ]:
!nvprof ./kernel2.out Input Output2 8 mask.txt

Started processing 8 images with dimensions : (1280, 720, 3) 
==1289== NVPROF is profiling process 1289, command: ./kernel2.out Input Output2 8 mask.txt
Batch 1 completed

Writing 8 outputs ... 
----------------------------------------------------------------------------------
==1289== Profiling application: ./kernel2.out Input Output2 8 mask.txt
==1289== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   73.54%  20.226ms         1  20.226ms  20.226ms  20.226ms  [CUDA memcpy DtoH]
                   16.73%  4.6019ms         2  2.3009ms  1.1840us  4.6007ms  [CUDA memcpy HtoD]
                    9.73%  2.6747ms         1  2.6747ms  2.6747ms  2.6747ms  convolution3D(unsigned char const *, int, int, int, float*, float const *, int)
      API calls:   85.39%  184.55ms         3  61.518ms  77.059us  184.36ms  cudaMalloc
                   11.99%  25.908ms         3  8.6360ms  74.952us  21.045ms  cudaMemcpy
                 

### Kernel 3

In [ ]:
!nvprof ./kernel3.out Input Output3 8 mask.txt

Started processing 8 images with dimensions : (1280, 720, 3) 
==1210== NVPROF is profiling process 1210, command: ./kernel3.out Input Output3 8 mask.txt
Batch 1 completed

Writing 8 outputs ... 
----------------------------------------------------------------------------------
==1210== Profiling application: ./kernel3.out Input Output3 8 mask.txt
==1210== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   74.12%  21.334ms         1  21.334ms  21.334ms  21.334ms  [CUDA memcpy DtoH]
                   15.87%  4.5667ms         2  2.2833ms  1.1520us  4.5655ms  [CUDA memcpy HtoD]
                   10.01%  2.8813ms         1  2.8813ms  2.8813ms  2.8813ms  convolution3D(unsigned char const *, int, int, int, float*, float const *, int)
      API calls:   85.92%  202.40ms         3  67.468ms  123.36us  202.07ms  cudaMalloc
                   11.50%  27.085ms         3  9.0283ms  71.828us  22.222ms  cudaMemcpy
                 